In [5]:

!pip install torchvision

     |████████████████████████████████| 4.0MB 7.0MB/s eta 0:00:01
     |████████████████████████████████| 753.4MB 14kB/s s eta 0:00:01              | 25.8MB 17.3MB/s eta 0:00:43     |█████████                       | 211.2MB 33.9MB/s eta 0:00:16     |██████████▋                     | 249.7MB 29.1MB/s eta 0:00:18�██████████▍                | 363.5MB 14.9MB/s eta 0:00:27     |█████████████████████████▌      | 600.8MB 41.7MB/s eta 0:00:04     |███████████████████████████▏    | 640.3MB 30.0MB/s eta 0:00:04


In [ ]:
# These are the libraries will be used for this lab.
import torchvision.models as models
from PIL import Image
import pandas
from torchvision import transforms
import torch.nn as nn
import time
import torch 
import matplotlib.pylab as plt
import numpy as np
from torch.utils.data import Dataset, DataLoader
import h5py
import os
import glob
torch.manual_seed(0)

In [ ]:
model=models.resnet18(pretrained=True)
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

composed=transforms.Compose([transforms.Resize(224),
                            transforms.ToTensor(),
                            transforms.Normalize(mean,std)])


In [ ]:
train_dataset=Dataset(transforms=composed, train=True)
validation_dataset=Dataset(transforms=composed)

In [ ]:
for param in model.parameters():
    param.requires_grad=False
   

In [ ]:
model.fc=nn.Linear(512,2)

In [ ]:
print(model)

# Data

In [ ]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/images/Positive_tensors.zip 
!unzip -q Positive_tensors.zip 

In [ ]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/images/Negative_tensors.zip
!unzip -q Negative_tensors.zip

In [ ]:
# Create your own dataset object

class Dataset(Dataset):

    # Constructor
    def __init__(self,transform=None,train=True):
        directory="/home/dsxuser/work"
        positive="Positive_tensors"
        negative='Negative_tensors'

        positive_file_path=os.path.join(directory,positive)
        negative_file_path=os.path.join(directory,negative)
        positive_files=[os.path.join(positive_file_path,file) for file in os.listdir(positive_file_path) if file.endswith(".pt")]
        negative_files=[os.path.join(negative_file_path,file) for file in os.listdir(negative_file_path) if file.endswith(".pt")]
        number_of_samples=len(positive_files)+len(negative_files)
        self.all_files=[None]*number_of_samples
        self.all_files[::2]=positive_files
        self.all_files[1::2]=negative_files 
        # The transform is goint to be used on image
        self.transform = transform
        #torch.LongTensor
        self.Y=torch.zeros([number_of_samples]).type(torch.LongTensor)
        self.Y[::2]=1
        self.Y[1::2]=0
        
        if train:
            self.all_files=self.all_files[0:30000]
            self.Y=self.Y[0:30000]
            self.len=len(self.all_files)
        else:
            self.all_files=self.all_files[30000:]
            self.Y=self.Y[30000:]
            self.len=len(self.all_files)     
       
    # Get the length
    def __len__(self):
        return self.len
    
    # Getter
    def __getitem__(self, idx):
               
        image=torch.load(self.all_files[idx])
        y=self.Y[idx]
                  
        # If there is any transform method, apply it onto the image
        if self.transform:
            image = self.transform(image)

        return image, y
    


In [ ]:
train_loader=torch.utils.data.DataLoader(dataset=train_dataset, batch_size=15)
validation_loader=torch.utils.data.DataLoader(dataset=validation_dataset, batch_size=10)

In [ ]:
criterion=nn.CrossEntropyLoss()

In [ ]:
optimizer = torch.optim.Adam([parameters  for parameters in model.parameters() if parameters.requires_grad],lr=0.001)

In [ ]:
n_epochs=10
loss_list=[]
accuracy_list=[]
correct=0
N_test=len(validation_dataset)
N_train=len(train_dataset)
start_time = time.time()
#n_epochs

Loss=0
start_time = time.time()
for epoch in range(n_epochs):
    for x, y in train_loader:

        model.train() 
        optimizer.zero_grad()
        z=model(x)
        loss=criterion(z,y)
        loss_sublist.append(loss.data.item())
        loss.backward()
        optimizer.step()
        loss_list.append(np.mean(loss_sublist))
                  
        loss_list.append(loss.data)
        
        
    correct=0
    for x_test, y_test in validation_loader:
        # set model to eval 
        model.eval()
        z=model(x_test)        
        _,yhat=torch.max(z.data,1)
        correct+=(yhat==y_test).sum().item()
       
        accuracy=correct/N_test
        accuracy_list.append(accuracy)

In [ ]:
to_pil = transforms.ToPILImage()
images, labels = get_random_images(4)
fig=plt.figure(figsize=(10,10))
for ii in range(len(images)):
    image = to_pil(images[ii])
    index = predict_image(image)
    sub = fig.add_subplot(1, len(images), ii+1)
    res = int(labels[ii]) == index
    sub.set_title(str(classes[index]) + ":" + str(res))
    plt.axis('off')
    plt.imshow(image)
plt.show()